# 検索データ作成

[Natsume](https://hinoki-project.org/natsume/)は，名詞ー格助詞ー動詞などの構文パターンを検索したり，そのジャンル間の使用を比較したりすることができるシステムです。
ここでは，その検索機能の位置，名詞ー格助詞ー動詞の構文パターンを抽出することにします。

## 準備

すでに`requirements.txt`を`pip`などでインストール済みだったら，以下の宣言は不要です。Google Colabなどの場合は実行が必要になります（`#`をとってください）。

DHラボのiMacはここで`pip3`を使ってください。

In [ ]:
![[ ! -d natsume-simple ]] && git clone https://github.com/borh/natsume-simple.git
![[ ! -d natsume-simple ]] && mv natsume-simple/* .
!pip install -r requirements.txt

## Ginza

Ginza は SpaCy を裏で使っているので，SpaCy と使用がほとんど変わりません。ただし，一部追加機能があります。
追加機能は主に文節処理とトーケン（形態素）のレマ（語彙素）参照です。詳しくは[公式サイトへ](https://megagonlabs.github.io/ginza/)。


In [ ]:
import spacy
import ginza

try:
    is_using_gpu = spacy.prefer_gpu()  # GPUがあれば，使う # GPU搭載ならここでコメントを外す
except:
    is_using_gpu = False

if is_using_gpu:
    print("Using GPU")

try:
    nlp = spacy.load("ja_ginza_electra")  # あればja_ginza_electraを使用
    model_name = "ja_ginza_electra"
except Exception:
    nlp = spacy.load("ja_ginza")  # なけらばja_ginza
    model_name = "ja_ginza"

ginza.force_using_normalized_form_as_lemma(True)

example_sentence = "東京では，銀座でランチを食べよう。"
doc = nlp(example_sentence)
[
    (
        token.i,
        token.orth_,
        token.lemma_,
        token.pos_,
        token.tag_,
        token.dep_,
        token.head.i,
        ginza.inflection(token),
    )
    for token in doc
]


## 係り受けの例

今回対象としている名詞ー格助詞ー動詞（NPV）パターンは係り受け構造の中で，どのように現れるか，簡単な例で示せます。
SpaCy/Ginza で使用される係り受け構造の定義は Universal Dependencies 2 をご参照ください。

上記の例でもわかるように，名詞ー格助詞ー動詞でナイーブな抽出を行うと，「東京で食べる」「銀座で食べる」「ランチを食べる」の３共起表現が抽出されます。ただし，その中の「東京で食べる」は実は「で」格助詞単独ではなく，「では」という連語の形で出現します。
goo 辞書の解説では[以下の通り](https://dictionary.goo.ne.jp/word/%E3%81%A7%E3%81%AF/)定義されています：

```
 で‐は の解説

［連語］
《断定の助動詞「だ」の連用形＋係助詞「は」》判断の前提を表す。…であるとすれば。…だと。「雨では中止になる」「彼ではだれも承知しないだろう」
《格助詞「で」＋係助詞「は」》…で。…においては。…を用いては。「今日では問題にされない」
《接続助詞「で」＋係助詞「は」》未然形に付く。
[...]
```

他にも「でも」「へと」「へは」「からは」など複合助詞が存在し，単独係助詞よりは文法的な役割が複雑なため，検索対象から外すようにします。

### 係り受け関係の可視化

Cabocha や KNP のように文節を係り受けの単位にしているものと違い，SpaCy/GiNZA ではトーケン（形態素）を単位として係り受け関係を表しています。
そのため，長文になればなるほど，その構造が最初の例（print 関数などを使う）より読みにくくなってしまいます。
そのため，SpaCy では可視化ツール displacy を用意しています。

よく使うので，最初にヘルパー関数 pp を定義し，文字列を入力として簡単にかかり受け図を出力するようにしておきます。


In [ ]:
from spacy import displacy

# https://spacy.io/api/top-level#displacy_options
compact = {
    "compact": True,
    "add_lemma": True,
    "distance": 100,
    "word_spacing": 30,
    "color": "#ffffff",
    "bg": "#1e1e1e",  # dark modeでない場合は，コメントアウト
}  # 表示を長い文用に工夫


def pp(s: str):
    return displacy.render(nlp(s), options=compact, jupyter=True)


pp("東京では，銀座でランチを食べよう。")

In [ ]:
# 残念ながら，y幅は調整できない
pp(
    "１８紀の哲学者ヒュームは，「力はいつも被治者の側にあり，支配者には自分たちを支えるものは世論以外に何もないということがわかるであろう」と論じているが，仮に選挙がなくとも，大多数の被治者からの暗黙の同意がなければ如何なる政治体制も不安定にならざるを得ないだろう。"
)


In [ ]:
pp(
    "さらに，数年おきに選挙が行われるだけではなく，マスメディアが発達し，世論調査が頻繁に行われている現在の状況を考えれば，以前と比べて，民意の重要性は，高まっていると思われる。"
)


## 係り受け関係の抽出

係り受け関係は SpaCy で DependencyMatcher という機能で検索できます。

- <https://spacy.io/usage/rule-based-matching#dependencymatcher>

Semgrex の記号を使うことによって，係り受け構造の定義がわりと自由にできます。

```
SYMBOL	DESCRIPTION
A < B	A is the immediate dependent of B.
A > B	A is the immediate head of B.
A << B	A is the dependent in a chain to B following dep → head paths.
A >> B	A is the head in a chain to B following head → dep paths.
A . B	A immediately precedes B, i.e. A.i == B.i - 1, and both are within the same dependency tree.
A .* B	A precedes B, i.e. A.i < B.i, and both are within the same dependency tree (not in Semgrex).
A ; B	A immediately follows B, i.e. A.i == B.i + 1, and both are within the same dependency tree (not in Semgrex).
A ;* B	A follows B, i.e. A.i > B.i, and both are within the same dependency tree (not in Semgrex).
A $+ B	B is a right immediate sibling of A, i.e. A and B have the same parent and A.i == B.i - 1.
A $- B	B is a left immediate sibling of A, i.e. A and B have the same parent and A.i == B.i + 1.
A $++ B	B is a right sibling of A, i.e. A and B have the same parent and A.i < B.i.
A $-- B	B is a left sibling of A, i.e. A and B have the same parent and A.i > B.i.
```

DependencyMatcher の利用が向いているのは，検索対象の型が固定であり，マッチングに否定が必要ない時です。
しかし，


In [ ]:
from spacy.matcher import DependencyMatcher


In [ ]:
from typing import Any, Optional


def token(name, attrs, dep_name=None, rel_op=None):
    spec = {
        "RIGHT_ID": name,
        "RIGHT_ATTRS": attrs,
    }
    if dep_name and rel_op:
        spec["LEFT_ID"] = dep_name
        spec["REL_OP"] = rel_op
    return spec


token("verb", {"POS": "VERB"})

In [ ]:
token("noun", {"DEP": {"IN": ["obj", "obl", "nsubj"]}})


In [ ]:
pattern = [
    # anchor token: VERB
    {"RIGHT_ID": "verb", "RIGHT_ATTRS": {"POS": "VERB"}},
    #
    {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": "noun",
        "RIGHT_ATTRS": {"DEP": {"IN": ["obj", "obl", "nsubj"]}},
    },
    #
    {
        "LEFT_ID": "noun",
        "REL_OP": ">",
        "RIGHT_ID": "case_particle",
        "RIGHT_ATTRS": {
            "DEP": "case",
            "LEMMA": {"IN": ["が", "を", "に", "で", "から", "より", "と", "へ"]},
        },
    },
]

from spacy.tokens import Doc


def matches_to_npv(doc: Doc, matches):
    exclude_matches: set[int] = set()
    for match_id, (verb, noun, case_particle) in matches:
        if doc[case_particle + 1].pos_ == "ADP":
            print(
                "Double particle:", doc[case_particle : case_particle + 2], "excluding."
            )
            exclude_matches.add(match_id)
    matches = [m for m in matches if m[0] not in exclude_matches]
    return matches


matcher = DependencyMatcher(nlp.vocab)
matcher.add("NPV", [pattern])
matches = matcher(doc)
matches = matches_to_npv(doc, matches)
matches


In [ ]:
# https://github.com/explosion/spaCy/blob/master/spacy/symbols.pyx
# GiNZA特有のシンボルcaseがないことに注意。文字列ではなく以下のようにsymbolを使うことで処理が若干早くなる。
from typing import Iterable
from spacy.symbols import (
    NOUN,
    PROPN,
    PRON,
    NUM,
    VERB,
    SYM,
    PUNCT,
    ADP,
    SCONJ,
    obj,
    obl,
    nsubj,
)
from spacy.tokens import Token, Span
from ginza import bunsetu_span, inflection
import ginza  # 他のメソッドなどを使う時
from itertools import takewhile, tee  # pairwiseがPython 3.10で登場
import re

from collections.abc import Iterator


def pairwise(iterable: Iterable[Any]):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


def simple_lemma(token: Token) -> str:
    if token.lemma_ == "為る":
        return "する"
    elif token.lemma_ == "居る":
        return token.orth_
    elif token.lemma_ == "成る":
        return token.orth_
    elif token.lemma_ == "有る":
        return token.orth_
    else:
        return token.lemma_


def normalize_verb_span(tokens):
    """動詞が入っている文節のトーケンを入力として，正規化された動詞の文字列を返す。
    現在「ます」「た」は除外とし，基本形に直す処理をしているが，完全にすべての活用の組み合わせに対応していな。"""
    clean_tokens = [
        token for token in tokens if token.pos not in {PUNCT, SYM}
    ]  # 。「」などが始め，途中，終わりに出現することがあるので除外
    clean_tokens = list(
        takewhile(
            lambda token: token.pos not in {ADP, SCONJ}
            and token.lemma_ not in {"から", "ため", "たり", "こと", "よう"},
            clean_tokens,
        )
    )  # いる>>と(ADP)<<いう，「いる>>から(SCONJ)<<」は品詞で除外すると「て」も除外される
    if len(clean_tokens) == 1:
        return simple_lemma(clean_tokens[0])

    normalized_tokens: list[Token] = []
    token_pairs: list[tuple[Token, Token]] = list(pairwise(clean_tokens))
    for i, (token, next_token) in enumerate(token_pairs):
        normalized_tokens.append(token)
        if next_token.lemma_ == "ます" or next_token.lemma_ == "た":
            if re.match(r"^(五|上|下|サ|.変格|助動詞).+", inflection(token)):
                # TODO: ませんでした
                break
            else:
                normalized_tokens.append(nlp("する")[0])
                break
        elif next_token.lemma_ == "だ":  # なら(ば)，説明する>>なら(lemma=だ)<<，
            break
        elif i == len(token_pairs) - 1:  # ペアが最後の場合はnext_tokenも格納
            normalized_tokens.append(next_token)

    if len(normalized_tokens) == 1:
        return simple_lemma(normalized_tokens[0])

    if not normalized_tokens:
        return None

    stem = normalized_tokens[0]
    affixes = normalized_tokens[1:-1]
    suffix = normalized_tokens[-1]
    return "{}{}{}".format(
        stem.text,  # .lemma_を使う場合は未然形・連用形など注意する必要あり
        "".join(t.text for t in affixes),
        simple_lemma(suffix),
    )


In [ ]:
[inflection(t) for t in nlp("語ります")]


### テストの活用

自然言語はその単語の組み合わせが膨大で，すべてをルールで記載するつもりが例外出てきて思わぬ結果になることが多いです。
ルールあるいはプログラムのアルゴリズム・処理などを検証しながら開発を進みたいときは，Python のテスト機能を活用とよいでしょう。
しばしば，ノートブックのセルでの実行結果を見ながら書くよりはテストにおさめて，いかなる変更で，以前できた処理ができなかったりする場合やほしい結果がどの時点で得られたかを早期発見できます。

以下では，全箇所が正しく処理されるのに対し，最後は失敗します。
（実際は「見られない」が正しいですが，失敗の例として「見られないが」を正解にしています。）


In [ ]:
import unittest


class TestVerbNormalization(unittest.TestCase):
    def test_norm(self):
        self.assertEqual(normalize_verb_span(nlp("いるからで")), "いる")
        self.assertEqual(normalize_verb_span(nlp("いるという")), "いる")
        self.assertEqual(normalize_verb_span(nlp("語ります")), "語る")
        self.assertEqual(normalize_verb_span(nlp("しました。")), "する")
        self.assertEqual(normalize_verb_span(nlp("作り上げたか")), "作り上げる")
        self.assertEqual(normalize_verb_span(nlp("見られなかったが")), "見られないが")

# ノートブックの中では以下のようにユニットテストできる：
unittest.main(
    argv=["ignored", "-v", "TestVerbNormalization.test_norm"], verbosity=2, exit=False
)


In [ ]:
# https://megagonlabs.github.io/ginza/bunsetu_api.html


def npv_matcher(doc: Doc):
    matches: list[tuple[str, str, str]] = []
    for token in doc[:-2]:  # 検索対象の最小トーケン数が３のため，最後の2トーケンは見なくて良い
        noun = token
        case_particle = noun.nbor(1)
        verb = token.head
        if (
            noun.pos in {NOUN, PROPN, PRON, NUM}
            and noun.dep in {obj, obl, nsubj}
            and verb.pos == VERB
            and case_particle.dep_ == "case"
            and case_particle.lemma_ in {"が", "を", "に", "で", "から", "より", "と", "へ"}
            and case_particle.nbor().dep_ != "fixed"
            and case_particle.nbor().head != case_particle.head
        ):  # では，には，をも，へとなどを除外
            verb_bunsetu_span = bunsetu_span(verb)
            vp_string = normalize_verb_span(verb_bunsetu_span)
            if not vp_string:
                print(
                    "Error normalizing verb phrase:",
                    verb_bunsetu_span,
                    "in document",
                    doc,
                )
                continue
            matches.append(
                (
                    noun.lemma_,
                    case_particle.lemma_,
                    # verb.lemma_,
                    vp_string,
                )
            )
    return matches


import unittest


class TestExtraction(unittest.TestCase):
    def test_npv(self):
        self.assertEqual(
            npv_matcher(nlp(example_sentence)),
            [("銀座", "で", "食べる"), ("ランチ", "を", "食べる")],
        )
        self.assertEqual(npv_matcher(nlp("京都にも行く。")), [])
        self.assertEqual(npv_matcher(nlp("ことを説明するならば")), [("こと", "を", "説明する")])
        # ここは「ことになる」あるいは「ことにならない」が正しいが，GiNZAではこれがイディオム処理（fixed/compound）のため，「ざるをえない」などと一緒に処理すべき
        self.assertEqual(npv_matcher(nlp("ことにならない")), [("こと", "に", "ならない")])


unittest.main(
    argv=["ignored", "-v", "TestExtraction.test_npv"], verbosity=2, exit=False
)


In [ ]:
npv_matcher(nlp("彼が語ります"))

In [ ]:
npv_matcher(nlp("ことを説明するならば"))

In [ ]:
npv_matcher(nlp("ことにならない"))

## TED トークコーパスの作成

Hugginface の datasets を使って，TED トークの日本語に翻訳された字幕をコーパス化します。
データセットのページは以下：

- <https://huggingface.co/datasets/ted_talks_iwslt>


In [ ]:
from datasets import load_dataset

ted_dataset_2014 = load_dataset(
    "ted_talks_iwslt", language_pair=("en", "ja"), year="2014"
)
ted_dataset_2015 = load_dataset(
    "ted_talks_iwslt", language_pair=("en", "ja"), year="2015"
)
ted_dataset_2016 = load_dataset(
    "ted_talks_iwslt", language_pair=("en", "ja"), year="2016"
)


In [ ]:
ted_corpus = (
    [d["ja"] for d in ted_dataset_2014["train"]["translation"]]
    + [d["ja"] for d in ted_dataset_2015["train"]["translation"]]
    + [d["ja"] for d in ted_dataset_2016["train"]["translation"]]
)
len(ted_corpus)


In [ ]:
ted_corpus


In [ ]:
with open("./data/ted_corpus.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(ted_corpus))


In [ ]:
pp("トビー・エクルズは、この状況を覆すための画期的なアイデア「ソーシャル・インパクト・ボンド（社会インパクト債権）」について話します。")


In [ ]:
pp("野生生物の保護に尽力するボイド・ヴァーティは「自然の大聖堂は人間性の最高の部分を映し出してくれる鏡である」と話します。")


In [ ]:
pp("チャンの素晴らしい手作りの弓がしなる様子をご堪能ください。")


In [ ]:
pp("アメリカ人が「共有する」市民生活は、どれだけお金を持っているかによって違うものになってしまったと言っていいでしょう。")


In [ ]:
pp("しかし飛行機や自動車が生まれて100年がたった今も、それが本当に実現されたことはありませんでした。")


In [ ]:
# バグ
pp("TEDxTC でジョナサン・フォーリーが「テラカルチャー」（地球全体のための農業）に取り組む必要性を訴えます。")


In [ ]:
pp(
    "感動のトーク　マッカーサー賞受賞者である活動家のマジョラ・カーターが サウスブロンクスの環境正義を求める闘いについて詳しく説明し 都市政策の欠陥  マイノリティ地区に最大の被害を受けることを示します"
)


In [ ]:
ginza.bunsetu_spans(
    nlp(
        "感動のトーク　マッカーサー賞受賞者である活動家のマジョラ・カーターが サウスブロンクスの環境正義を求める闘いについて詳しく説明し 都市政策の欠陥  マイノリティ地区に最大の被害を受けることを示します"
    )
)


In [ ]:
npv_matcher(
    nlp(
        "感動のトーク　マッカーサー賞受賞者である活動家のマジョラ・カーターが サウスブロンクスの環境正義を求める闘いについて詳しく説明し 都市政策の欠陥  マイノリティ地区に最大の被害を受けることを示します"
    )
)


## コーパスからの抽出処理

処理する文章が多いときは`nlp.pipe()`を使い，文字列のリストを引数にすることで，並列処理が行えます。
そこから得られた doc(s) を npv_matcher に渡し，chain.from_iterable でくっつけます。


In [ ]:
from itertools import chain

ted_npvs = list(chain.from_iterable(npv_matcher(doc) for doc in nlp.pipe(ted_corpus)))
# GPU 25s   (ja_ginza) / 1m3s  (ja_ginza_electra) (A4000)
# CPU 1m31s (ja_ginza) / 5m42s (ja_ginza_electra) (3960x)


In [ ]:
len(ted_npvs)


In [ ]:
# 格助詞ごとの項目数を調べるなら
from collections import Counter

Counter(npv[1] for npv in ted_npvs)


In [ ]:
import pandas as pd


## NPV データの保存

検索インターフェースでは今回 NPV パターンのみを検索するため，そのデータのみを CSV 形式に書き出す。


In [ ]:
from pathlib import Path

data_dir = Path("./data/")
data_dir


In [ ]:
df = pd.DataFrame.from_records(ted_npvs, columns=["n", "p", "v"])
df["corpus"] = "TED"
df


In [ ]:
df.to_csv(data_dir / f"ted_npvs_{model_name}.csv", index=False)


In [ ]:
# Google Colab has an old version of Pandoc, so we download and install the latest release.
!wget -c https://github.com/jgm/pandoc/releases/download/2.19.2/pandoc-2.19.2-1-amd64.deb
!sudo dpkg -i pandoc-2.19.2-1-amd64.deb
# Using this Pandoc, the following scripts should work: (Bash and Python versions are interchangeable)
!cd scripts && ./get-jnlp-corpus.sh
!cd scripts && python ./convert-jnlp-corpus.py

In [ ]:
with open(data_dir / "jnlp-sample-3000-python.txt", encoding="utf-8") as f:
    jnlp_corpus = f.readlines()

jnlp_npvs = list(chain.from_iterable(npv_matcher(doc) for doc in nlp.pipe(jnlp_corpus, disable=["ner"], batch_size=100, n_process=2)))
# GPU 28s   (ja_ginza) / 1m6s (ja_ginza_electra) (A4000)
# CPU 1m42s (ja_ginza) / 6m2s (ja_ginza_electra) (3960x)


In [ ]:
j_df = pd.DataFrame.from_records(jnlp_npvs, columns=["n", "p", "v"])
j_df["corpus"] = "自然言語処理"
j_df.to_csv(f"./data/jnlp_npvs_{model_name}.csv", index=False)
j_df


In [ ]:
pp(
    "共参照関係認定基準1を用いた場合と共参照関係認定基準2を用いた場合とを比較すると，共参照関係認定基準2の方が厳しい制約であるため，再現率が低下するかわりに，適合率が上昇している．"
)
